### The Problem
Misal kita punya dua tugas. Kita harus menulisa pro dan konta dari suatu produk. Untuk menjalankannya secara linear, pertama harus kita buat yang pro terlebih dahulu (makan waktu 30s) dan kemudian melanjutkannya dengan yang kontra (makan waktu 30s lagi). Dengan cara linear ini membutuhkan waktu sebanyak 1min.

Tapi..

Jika kita melakukan tugasnya itu bersamaan maka tidak akan memakan waktu sebanyak itu. Sehingganya metode paralel lebih baik.

### Studi Kasus

Kita akan membuat chain untuk menggenerate kelebihan dan kekurangan dari sebuah produk.

In [47]:
from langchain_groq import ChatGroq
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableParallel
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv

load_dotenv()

True

In [48]:
model = ChatGroq(model="llama3-8b-8192")

**#1 Buat prompt template awal**

In [49]:
# Prompt Awal
prompt_awal = ChatPromptTemplate.from_messages([
    ("system", "Kamu adalah seorang expert product reviewer"),
    ("human", "Tolong buat list dari fitur utama pada produk {produk}")
])

**#2 Membuat fungsi menganalisa kelebihan dan kekurangan**

Tiap fungsi ini akan dibuatkan prompt baru untuk modelnya gunakan sesuai dengan tujuan fungsinya.

In [60]:
# Fungsi analisa kelebihan produk
def analisa_kelebihan(fitur):
    # Membuat prompt template untuk menganalisa kelebihan
    pro_template = ChatPromptTemplate.from_messages([
        ("system", "Kamu adalah seorang expert product reviewer"),
        ("human", "Berikan fitur ini: {fitur}, list dari kelebihan fitur yang dimiliki")
    ])
    
    return pro_template.format_prompt(fitur=fitur).to_messages()

# Fungsi analisa kekurangan produk
def analisa_kekurangan(fitur):
    # Membuat prompt template untuk menganalisa kekurangan
    cons_prompt = ChatPromptTemplate.from_messages([
        ("system", "Kamu adalah seorang expert product reviewer"),
        ("human", "Berikan fitur ini: {fitur}, list dari kekurangan fitur yang dimiliki")
    ])
    
    return cons_prompt.format_prompt(fitur=fitur).to_messages()

**#3 Membuat fungsi untuk menggabungkan kelebihan dan kekurangan**

Tujuannya untuk membuat dictionary dari chain yang dibuat

In [51]:
def gabung_pros_cons(pros, cons):
    return f"Kelebihan:\n{pros}\n\nKekurangan:\n{cons}"

**#4 Membuat RunnableLambda untuk mensimplifikasi cabang-cabang (branches) dengan menggunakan LCEL** 

In [63]:
pros_branch = (
    # x adalah fitur. analisa_kelebihan(x) akan mengembalikan prompt template yang akan digunakan model
    RunnableLambda(lambda x: analisa_kelebihan(x)) | model | StrOutputParser()
)

cons_branch = (
    RunnableLambda(lambda x: analisa_kekurangan(x)) | model | StrOutputParser()
)

**#5 Membuat kombinasi chain menggunakan LCEL**

In [68]:
chain = (
    prompt_awal
    | model
    | StrOutputParser()
    
    # Menggunakan RunnableParallel
    | RunnableParallel(branches={"kelebihan": pros_branch, "kekurangan": cons_branch})
    
    # Menyatukan kembali semua branch itu
    | RunnableLambda(lambda x: gabung_pros_cons(x["branches"]["kelebihan"], x["branches"]["kekurangan"]))
)

In [69]:
result = chain.invoke({"produk": "MakBook Pro"})
print(result)

Kelebihan:
Based on the features listed, here are the benefits of the MacBook Pro:

1. **Display**:
	* High-resolution Retina display with wide color gamut for vibrant colors
	* Anti-reflective coating for reduced glare
	* 500 nits of brightness for improved visibility
2. **Performance**:
	* Up to 8th generation Intel Core i9 processor with Turbo Boost up to 5.0 GHz
	* Up to 64GB of DDR4 RAM for smooth multitasking
	* Fast storage options with read speeds up to 3.2GB/s
3. **Graphics**:
	* High-performance AMD Radeon Pro graphics with 4GB or 8GB of GDDR5 memory
	* Option for Intel Iris Plus Graphics 655 for improved graphics performance
4. **Battery Life**:
	* Up to 10 hours of battery life for extended use
	* Fast charging capabilities for quick top-ups
5. **Portability**:
	* Lightweight and compact design, starting at 3.02 pounds (13-inch) and 4.3 pounds (15-inch)
	* Aluminum unibody construction for durability
6. **Sound**:
	* High-fidelity audio with wider stereo sound
	* Stereo spe

---


### Catatan Error: **LangChain Chain Parallel**

#### **Deskripsi Masalah**
Saat mencoba membuat sebuah chain paralel menggunakan **LangChain**, error berikut muncul:

```shell
TypeError: Expected a Runnable, callable or dict. Instead got an unsupported type: <class 'langchain_core.prompt_values.ChatPromptValue'>
```

Error ini terjadi ketika mencoba menjalankan `RunnableParallel` dengan branch yang mengembalikan tipe data **ChatPromptValue**. Namun, `RunnableParallel` mengharapkan `Runnable`, callable, atau dictionary.

#### **Penyebab**
1. **Tipe Data Tidak Didukung:**  
   Fungsi `analisa_kelebihan` dan `analisa_kekurangan` mengembalikan objek `ChatPromptValue` dari `ChatPromptTemplate`. Tipe ini tidak dikenali langsung oleh LangChain sebagai input yang valid untuk cabang paralel.

2. **Kesalahan Penanganan Output Prompt:**  
   Output dari fungsi `analisa_kelebihan` dan `analisa_kekurangan` belum diubah menjadi format yang sesuai sebelum diproses lebih lanjut oleh model.

```python
   def analisa_kelebihan(fitur):
      # ...      
      return pro_template.format_prompt(fitur=fitur)  # Ini mengembalikan tipe data ChatPromptTemplate

   def analisa_kekurangan(fitur):
      # ...      
      return cons_prompt.format_prompt(fitur=fitur)   # Ini mengembalikan tipe data ChatPromptTemplate
```

#### **Langkah Penyelesaian**
Untuk mengatasi error ini, berikut langkah-langkah yang dilakukan:

1. **Ubah Output Prompt ke Format yang Sesuai:**
   Gunakan `.to_messages()` setelah `format_prompt()` agar keluaran berupa format yang diharapkan model.

2. **Panggil Model dengan Benar di Lambda:**
   Pastikan model dipanggil menggunakan `.invoke()` di dalam `RunnableLambda` dengan input yang sudah sesuai.

3. **Sesuaikan Cabang `RunnableParallel`:**
   Perbaiki setiap cabang (`pros_branch` dan `cons_branch`) agar memproses input dan output dengan benar.

#### **Kode yang Telah Diperbaiki**
Berikut adalah kode yang telah diperbaiki:

```python
# Fungsi analisa kelebihan produk
def analisa_kelebihan(fitur):
    # Membuat prompt template untuk menganalisa kelebihan
    pro_template = ChatPromptTemplate.from_messages([
        ("system", "Kamu adalah seorang expert product reviewer"),
        ("human", "Berikan fitur ini: {fitur}, list dari kelebihan fitur yang dimiliki")
    ])
    # Pastikan hasilnya berupa dictionary yang siap digunakan model
    return pro_template.format_prompt(fitur=fitur).to_messages()  # Ini baru benar 👍🏻

# Fungsi analisa kekurangan produk
def analisa_kekurangan(fitur):
    # Membuat prompt template untuk menganalisa kekurangan
    cons_prompt = ChatPromptTemplate.from_messages([
        ("system", "Kamu adalah seorang expert product reviewer"),
        ("human", "Berikan fitur ini: {fitur}, list dari kekurangan fitur yang dimiliki")
    ])
    # Pastikan hasilnya berupa dictionary yang siap digunakan model
    return cons_prompt.format_prompt(fitur=fitur).to_messages()   # Ini baru benar 👍🏻

# RunnableLambda cabang kelebihan
pros_branch = (
    RunnableLambda(lambda x: model.invoke(analisa_kelebihan(x)))
    | StrOutputParser()
)

# RunnableLambda cabang kekurangan
cons_branch = (
    RunnableLambda(lambda x: model.invoke(analisa_kekurangan(x)))
    | StrOutputParser()
)

# Gabungan chain
chain = (
    prompt_awal
    | model
    | StrOutputParser()
    | RunnableParallel(branches={"kelebihan": pros_branch, "kekurangan": cons_branch})
    | RunnableLambda(lambda x: gabung_pros_cons(x["kelebihan"], x["kekurangan"]))
)

# Eksekusi
result = chain.invoke({"produk": "MacBook Pro"})
print(result)
```

#### **Penjelasan Perbaikan**
1. **Konversi Output Prompt:**  
   `format_prompt()` diubah menjadi `format_prompt().to_messages()` agar hasilnya berupa daftar pesan yang dikenali model.

2. **Model Invocation:**  
   Model dipanggil secara langsung di dalam `RunnableLambda` dengan `model.invoke()`.

3. **Struktur Chain:**  
   Semua cabang (`kelebihan` dan `kekurangan`) diproses secara paralel menggunakan `RunnableParallel` dan hasilnya digabungkan kembali di tahap akhir.

#### **Hasil Akhir**
Kode berhasil berjalan tanpa error, dan output dari chain paralel dapat diproses dengan benar.

--- 
Catatan ini dapat digunakan sebagai referensi untuk troubleshooting jika masalah serupa muncul di masa mendatang. 😊